In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import networkx as nx
import torch
import torch_geometric as tg
import numpy as np

from utils.transforms import EuclideanInformationTransform, OneHot
from  models.se3_attention_mechanisms import Se3EquivariantAttentionMechanism

import e3nn

### Let's test the SE3 equivariant version of the GAT

In [3]:
g = nx.DiGraph()

vertices = (0, 1, 2)
edges = [(0, 1),
         (1, 0),
         (1, 2),
         (2, 0),
         ]

z = [0, 1, 2,]
pos = [(0.,   0.,  0.),
       (-1., -1., -1.),
       (1.,   1.,  1.),
     ]

features = {i: {'z': z[i], 'pos': pos[i]} for i in vertices }

for v in vertices:
    g.add_node(v)

for e in edges:
    g.add_edge(*e)

nx.set_node_attributes(g, features)

graph = tg.utils.from_networkx(g)

euc_transform = EuclideanInformationTransform()
one_hot_transform = OneHot('z', 'z')
transform = tg.transforms.Compose([euc_transform, one_hot_transform])

graph = transform(graph)


In [4]:
test_dataloader = tg.data.DataLoader([graph, graph.clone()], batch_size=1)
for batch in test_dataloader:
    break

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [5]:
feature_irreps = e3nn.o3.Irreps("10x0e + 10x1e + 10x2e")
geometric_irreps = e3nn.o3.Irreps("3x0e+3x1e+3x2e")
output_irreps = e3nn.o3.Irreps("10x0e + 10x1e + 10x2e")
internal_key_query_irreps = e3nn.o3.Irreps("5x0e + 5x1e + 5x2e")


net = Se3EquivariantAttentionMechanism(
    feature_irreps=feature_irreps,
    geometric_irreps=geometric_irreps,
    value_out_irreps=output_irreps,
    key_and_query_out_irreps=internal_key_query_irreps,
    radial_network_hidden_units=16
)


C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\torch\jit\_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [6]:
from tests.test_attention_mechanism_equivariance import GraphInputEquivarianceTest
alpha, beta, gamma = e3nn.o3.rand_angles(1)

In [7]:
# For normal inputs
embed = torch.nn.Linear(batch.z.shape[1], feature_irreps.dim)
features = embed(batch.z.float())
edge_harmonics = e3nn.o3.spherical_harmonics(geometric_irreps,
                                             batch.relative_positions,
                                             normalize=False
                                             )

output = net.forward(edge_index=batch.edge_index,
           features=features,
           edge_features=edge_harmonics,
           distances=batch.distances,
           )

rotation_matrix_for_output = output_irreps.D_from_angles(alpha, beta, gamma)
rotated_output = output @ rotation_matrix_for_output

In [14]:
# Alternative way of getting the rotated harmonics
edge_harmonic_rotater = geometric_irreps.D_from_angles(alpha, beta, gamma).squeeze(0)
rotated_edge_harmonics =  edge_harmonics @ edge_harmonic_rotater

feature_rotater = feature_irreps.D_from_angles(alpha, beta, gamma).squeeze(0)
rotated_features = features @ feature_rotater

output_from_rotated = net.forward(
           edge_index=batch.edge_index,
           features=rotated_features,
           edge_features=rotated_edge_harmonics,
           distances=batch.distances,
           )


In [16]:
(rotated_output - output_from_rotated).max()

tensor(2.2054e-06, grad_fn=<MaxBackward1>)

In [10]:
# For rotated inputs
rotated_graph = batch.clone()
rotation_matrix = e3nn.o3._rotation.angles_to_matrix(alpha, beta, gamma)
rotated_graph.relative_positions = (rotation_matrix @ rotated_graph.relative_positions.unsqueeze(-1)).squeeze(-1)

edge_harmonics_from_rotated = e3nn.o3.spherical_harmonics(geometric_irreps,
                                                     rotated_graph.relative_positions,
                                                     normalize=False)
